<a href="https://colab.research.google.com/github/TomographicImaging/gVXR-Tutorials/blob/main/notebooks/first_xray_simulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# -*- coding: utf-8 -*-
#
#  Copyright 2024 United Kingdom Research and Innovation
#
#  Licensed under the Apache License, Version 2.0 (the "License");
#  you may not use this file except in compliance with the License.
#  You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
#  Unless required by applicable law or agreed to in writing, software
#  distributed under the License is distributed on an "AS IS" BASIS,
#  WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#  See the License for the specific language governing permissions and
#  limitations under the License.
#
#   Authored by:    Franck Vidal (UKRI-STFC)

![gVXR](https://github.com/TomographicImaging/gVXR-Tutorials/blob/main/img/Logo-transparent-small.png?raw=1)

# First simulated X-ray image

This notebook is a step-by-step tutorial to show how to create our first X-ray radiograph.
A mono-material object is imaged with a monochromatic source and an ideal detector.
We show how to visualise the X-ray radiograph and take a screenshot of the 3D visualisation of the simulation environment.

<div class="alert alert-block alert-warning">
    <b>Note:</b> Make sure the Python packages are already installed. See <a href="../README.md">README.md</a> in the root directory of the repository. If you are running this notebook from Google Colab, please run the cell below to install the package with `!pip install gvxr`
</div>

In [ ]:
import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    !pip install gvxr

# Aims of this session

1. Create our first X-ray simulation, step-by-step;
2. Save our X-ray image in a file format that preserves the original dynamic range;
3. Visualise the results with 3 different look-up tables;
4. Visualise the 3D environment.

![](https://github.com/TomographicImaging/gVXR-Tutorials/blob/main/img/02-visualisation.png?raw=1)

# Main steps

There are 6 main steps to simulate an X-ray image:

1. Create a renderer (OpenGL context)
2. Set the X-ray source
3. Set the Spectrum
4. Set the Detector
5. Set the Sample
6. Compute the corresponding X-ray image.

# Cheat Sheet

## Simulation parameters

| Parameters | Values | Units | Function call |
|------------|--------|-------|---------------|
| Source position | [-40, 0, 0] | cm | `gvxr.setSourcePosition(-40.0, 0.0, 0.0, "cm")` |
| Source type (beam shape) | Point source (cone beam) | | `gvxr.usePointSource()` |
| Beam spectrum | Monochromatic: 1,000 photons of 80 | keV | `gvxr.setMonoChromatic(80.0, "keV", 1000)` or |
| | | MeV | `gvxr.setMonoChromatic(0.08, "MeV", 1000)` |
| Detector position | [10, 0, 0] | cm | `gvxr.setDetectorPosition(10.0, 0.0, 0.0, "cm")` |
| Detector orientation | [0, 0, -1] |  | `gvxr.setDetectorUpVector(0, 0, -1)` |
| Detector resolution | 640 &times; 320 | pixels | `gvxr.setDetectorNumberOfPixels(640, 320)` |
| Pixel spacing | 0.5 &times; 0.5 | mm | `gvxr.setDetectorPixelSize(0.5, 0.5, "mm")` |
| Sample | STL file of the Welsh dragon | mm | `gvxr.loadMeshFile(string_ID, fname, "mm")` |
| Material composition | Ti90Al6V4 alloy |  | `gvxr.setMixture("Dragon", "Ti90Al6V4")` or |
| | | | `gvxr.setMixture("Dragon", [22, 13, 23], [0.9, 0.06, 0.04])` |
| Material density | 4.43 | g/cm3 | `gvxr.setDensity("Dragon", 4.43, "g/cm3")` or |
| | | g.cm-3 | `gvxr.setDensity("Dragon", 4.43, "g.cm-3")` |

## Other gVXR's function used

| Functions | Descriptions |
|-----------|--------------|
| `gvxr.createOpenGLContext` | Create a simulation environment automatically. You must call either `gvxr.createOpenGLContext()` or `gvxr.createNewContext()` before any other functions from gVXR. |
| `gvxr.moveToCentre` | Move a polygon mesh to the centre. |
| `gvxr.setElement` | Set the chemical element (e.g. iron) corresponding to the material properties of a polygon mesh. |
| `gvxr.setCompound` | Set the compound (e.g. water) corresponding to the material properties of a polygon mesh. |
| `gvxr.computeXRayImage` | Compute the X-ray projection corresponding to the environment that has previously been set. |
| `gvxr.displayScene` | Update the visualisation window. It works with both the interactive window and offscreen rendering. |
| `gvxr.saveLastXRayImage` | Save the last computed X-ray image in a file (e.g. TIFF, MHA, MHD/RAW) and store the data using single-precision floating-point numbers. |
| `gvxr.saveLastLBuffer` | Save the last computed path length in a file (e.g. TIFF, MHA, MHD/RAW) and store the data using single-precision floating-point numbers. The red channel is the path length; the green and blue channels can be used to detect errors (0.0 values mean no error). |
| `gvxr.setColour` | Change the colour of an object in the 3D visualisation. |
| `gvxr.setWindowBackGroundColour` | Change the background colour of the 3D visualisation. |
| `gvxr.takeScreenshot` | Take the screenshot of the current state of the 3D visualisation. |
| `gvxr.renderLoop` | Activate the interactive mode of the 3D visualisation. |
| `gvxr.terminate` | Close and destroy all the windows and simulation contexts that have been created. No further gVXR's function should be called after `gvxr.terminate`. |

## Import packages

- `os` to create the output directory if needed
- `matplotlib` to show 2D images
- `tifffile` to write TIFF files
- `gvxr` to simulate X-ray images

In [ ]:
# Import packages
import os # Create the output directory if necessary
import numpy as np # Who does not use Numpy?

import matplotlib # To plot images
import matplotlib.pyplot as plt # Plotting
from matplotlib.colors import LogNorm # Look up table
from matplotlib.colors import PowerNorm # Look up table

font = {'family' : 'serif',
         'size'   : 15
       }
matplotlib.rc('font', **font)

# Uncomment the line below to use LaTeX fonts
# matplotlib.rc('text', usetex=True)

from tifffile import imwrite # Write TIFF files

from gvxrPython3 import gvxr # Simulate X-ray images

Make sure the output directory exists

In [ ]:
output_path = "../notebooks/output_data/first_xray_image"
if not os.path.exists(output_path):
    os.makedirs(output_path);

## 1. Create an OpenGL context

The first step is to create the simulation environment, known here as "OpenGL context".
`createOpenGLContext` will try to find the most suitable environment possible regardless of the operating system. This is an alternative function to `gvxr.createNewContext` used in [test_installation.ipynb](test_installation.ipynb).

In [ ]:
# Create an OpenGL context
print("Create an OpenGL context");
gvxr.createOpenGLContext();

## 2. Set the X-ray source

We must set it's position and beam shape.

In [ ]:
# Create a source
print("Set up the beam");
gvxr.setSourcePosition(-40.0,  0.0, 0.0, "cm");
gvxr.usePointSource();
#  For a parallel source, use gvxr.useParallelBeam();

## 3. Set the Spectrum

We define here the number of photons and their kinetic energy.

In [ ]:
# Set its spectrum, here a monochromatic beam
# 1000 photons of 80 keV (i.e. 0.08 MeV) per ray
gvxr.setMonoChromatic(0.08, "MeV", 1000);
# The following is equivalent: gvxr.setMonoChromatic(80, "keV", 1000);

## 4. Set the Detector

A detector is defined by its position, orientation, pixel resolution and the space between the centre of two consecutive pixels along its two axes.

In [ ]:
# Set up the detector
print("Set up the detector");
gvxr.setDetectorPosition(10.0, 0.0, 0.0, "cm");
gvxr.setDetectorUpVector(0, 0, -1);
gvxr.setDetectorNumberOfPixels(640, 320);
gvxr.setDetectorPixelSize(0.5, 0.5, "mm");

## 5. Set the Sample

A sample is define by its geometry (surface) and material composition. Note that you can transform (translate, scale and rotate) a sample.

In [ ]:
# Locate the sample STL file from the package directory
path = os.path.dirname(gvxr.__file__);
fname = path + "/welsh-dragon-small.stl";

# Load the sample data
if not os.path.exists(fname):
    raise IOError(fname);

print("Load the mesh data from", fname);
gvxr.loadMeshFile("Dragon", fname, "mm");

print("Move ", "Dragon", " to the centre");
gvxr.moveToCentre("Dragon");

In [ ]:
# Material properties
print("Set ", "Dragon", "'s material");

# Iron (Z number: 26, symbol: Fe)
gvxr.setElement("Dragon", 26);
gvxr.setElement("Dragon", "Fe");

# Liquid water
gvxr.setCompound("Dragon", "H2O");
gvxr.setDensity("Dragon", 1.0, "g/cm3");
gvxr.setDensity("Dragon", 1.0, "g.cm-3");

# Titanium Aluminum Vanadium Alloy
gvxr.setMixture("Dragon", "Ti90Al6V4");
gvxr.setMixture("Dragon", [22, 13, 23], [0.9, 0.06, 0.04]);
# gvxr.setMixture("Dragon", ["Ti", "Al", "V"], [0.9, 0.06, 0.04]); # Not yet implemented
gvxr.setDensity("Dragon", 4.43, "g/cm3");
gvxr.setDensity("Dragon", 4.43, "g.cm-3");

## 6. Compute the corresponding X-ray image.

It is possible to compute, retrieve and save an X-ray image as well as the path length of X-ray through an object.

In [ ]:
# Compute an X-ray image
# We convert the array in a Numpy structure and store the data using single-precision floating-point numbers.
print("Compute an X-ray image");
x_ray_image = gvxr.computeXRayImage();

# Update the visualisation window
gvxr.displayScene();

In [ ]:
# Save the X-ray image in a TIFF file and store the data using single-precision floating-point numbers.
gvxr.saveLastXRayImage(os.path.join(output_path, 'raw_x-ray_image-01.tif'));

# The line below will also works
imwrite(os.path.join(output_path, 'raw_x-ray_image-02.tif'), x_ray_image);

# Save the L-buffer
gvxr.saveLastLBuffer(os.path.join(output_path, 'path_length-dragon.tif'));

A linear look up table is not always the best way to visualise X-ray images. A log-scale may be favoured (but be mindful $\log(0)$ does not exist. Another popular method is the Power-law colour scale as we can control the Power law exponent ($\gamma$). The corresponding normalisation formula is as follows:
$$\left(\frac{Img - \min(Img)}{\max(Img) - \min(img)}\right)^\gamma$$

In [ ]:
# Display the X-ray image and compare three different lookup tables
plt.figure(figsize=(15, 7.5));

plt.suptitle("Image simulated with gVirtualXray visualised", y=0.75);

plt.subplot(131);
plt.imshow(x_ray_image, cmap="gray");
plt.colorbar(orientation='horizontal');
plt.title("using a linear colour scale");

plt.subplot(132);
plt.imshow(x_ray_image, norm=LogNorm(), cmap="gray");
plt.colorbar(orientation='horizontal');
plt.title("using a logarithmic colour scale");

plt.subplot(133);
plt.imshow(x_ray_image, norm=PowerNorm(gamma=1./2.), cmap="gray");
plt.colorbar(orientation='horizontal');
plt.title("using a Power-law colour scale ($\gamma=0.5$)");

plt.tight_layout();
plt.savefig(os.path.join(output_path, 'projection.pdf'), dpi=600);
plt.show();

It may be useful to visualise the 3D environment to ascertain everything is as expected. It can be done in a window or offscreen with a screenshot.

In [ ]:
# Change the sample's colour
# By default the object is white, which is not always pretty. Let's change it to purple.
red = 102.0 / 255.0;
green = 51.0 / 255.0;
blue = 153.0 / 255.0;
gvxr.setColour("Dragon", red, green, blue, 1.0);

# This image can be used in a research paper to illustrate the simulation environment, in which case you may want to change the background colour to white with:
gvxr.setWindowBackGroundColour(1.0, 1.0, 1.0);

# Update the visualisation window
gvxr.displayScene();

In [ ]:
# Take the screenshot and save it in a file
screenshot = gvxr.takeScreenshot();
plt.imsave(os.path.join(output_path, "screenshot.png"), np.array(screenshot));

# or display it using Matplotlib
plt.figure(figsize=(10, 10));
plt.imshow(screenshot);
plt.title("Screenshot of the X-ray simulation environment");
plt.axis('off');
plt.show();

If windowing is possible, it is possible to run the 3D visualisation in an interactive window.

In [ ]:
# Interactive visualisation
# The user can rotate the 3D scene and zoom-in and -out in the visualisation window.

# - Keys are:
#     - Q/Escape: to quit the event loop (does not close the window)
#     - B: display/hide the X-ray beam
#     - W: display the polygon meshes in solid or wireframe
#     - N: display the X-ray image in negative or positive
#     - H: display/hide the X-ray detector
# - Mouse interactions:
#     - Zoom in/out: mouse wheel
#     - Rotation: Right mouse button down + move cursor```
gvxr.renderLoop();

# Cleaning up

Once we have finished, it is good practice to clean up the OpenGL contexts and windows with the following command. Note that due to the object-oriented programming nature of the core API of gVXR, this step is automatic anyway.

In [ ]:
# Works on Windows (gvxr >= 2.0.9)
# Will work on Linux (gvxr >= 2.0.10)
if os.name == 'nt':
    gvxr.destroy(); 
# Does not work on Windows.
# Deprecated, for backward compatibility on Linux.
else:
    gvxr.terminate(); 